# Modelo de Previsão de Demanda
**Autora:** Gué Cardoso

Sabemos que num mercado competitivo, é sempre importante estarmos à frente dos nossos concorrentes. E para garantir certa vantagem, seria de bom tom **alinhar a necessidades dos clientes**, **estoque** e **demanda de mercado**. Aí que os modelos de previsão de demanda entram. Tome como exemplo o Natal ou até mesmo as semanas que antecedem esta data. Com base só na nossa experiência, sabemos que algo muda ali, que o mercado fica muito mais aquecido e a procura por certos produtos aumentam consideravelmente. 


E ao invés de tomarmos decisões estratégicas baseada nos *achismos* ou as experiências vividas no passar dos anos, não seria mais seguro entender sua base histórica e criar mecanismos que auxiliem na tomada de decisão de forma a impactar no resultado final? O Forecasting não só é útil na parte de controle de estoque, mas também pode trazer uma visão sobre comportamento de compra, informações sobre a jornada de compra dos clientes.


Nas próximas linhas será apresentada uma solução simples para o problema de Store Sales Forecasting tendo como base 45 lojas do Walmart alocadas em diferentes regiões, cada uma contendo seus departamentos, suas promoções, seus tamanhos, etc. O desafio? Criar predições de vendas semanais.


A ideia inicial foi de criar um **modelo base** e a partir dele utiliar técnicas de **feature engineering** para melhorar tal modelo, pensando num esquema de CRISP-DM

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Imports das bibliotecas que serão utilizadas

In [ ]:
import pickle
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('Solarize_Light2') #plt.style.available
%matplotlib inline


from sklearn import metrics 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 

path = "../input/walmart-recruiting-store-sales-forecasting"


## Definição de funções 
* **TODO** criar arquivo .py com todas as funções, importar junto com as outras bibliotecas e chamar as funções no pipeline

In [ ]:
def read_create_merge_data(filelist):
    '''
    função que faz a leitura dos dados do diretório
    e cria um dataframe único fazendo merging dos dados
    de loja, treino e features
    '''
    data = pd.read_csv(filelist[0])
    aux =  pd.read_csv(filelist[1])
    data = data.merge(aux, how='left', on='Store')
    
    aux =  pd.read_csv(filelist[2])
    data = data.merge(aux, how='left', on=['Store','Date','IsHoliday'])
    
    return data


def df_filter(dataframe, col_name,col_value):
    '''função para filtro do dataframe'''
    return dataframe[dataframe[col_name]==col_value].reset_index(drop=True)


def create_rf_model(X,y):
    #Create model
    regr = RandomForestRegressor(n_estimators=100, random_state=42)
    regr.fit(X_train,y_train)
    return regr


def save_model(pkl_filename, rf_model):
    '''função para salvar o modelo em disco'''
    # saving the trained model to disk 
    pickle.dump(rf_model, open(pkl_filename, 'wb'))
    return "Saved model to disk"


def treat_data(df):
    '''
    função para extrair ano, mes, dia e semana do campo data
    '''
    df[['year','month','day']] = df['Date'].str.split('-',expand=True)
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)
    df['week'] = (((df['month']-1) * 30) + df['day'])//7
    return df



def plot_heatmap(dataframe):
    sns.heatmap(dataframe.corr(),annot=True,cmap='winter_r',linewidths=0.2) 
    fig=plt.gcf()
    fig.set_size_inches(13,9)
    return plt.show()


def plot_bar(dataframe, xCol_name, yCol_name):
    plt.figure(figsize=(20,8))
    sns.barplot(x=xCol_name, y=yCol_name, data=dataframe)
    plt.grid()
    plt.title('Média '+str(yCol_name), fontsize=18)
    plt.ylabel(xCol_name, fontsize=16)
    plt.xlabel(yCol_name, fontsize=16)
    return plt.show()


def get_holidays_df():
    '''cria dataframe com os feriados fornecidos'''
    holidays = {
    'Super Bowl': ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'],
    'Labor Day': ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'],
    'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'],
    'Christmas': ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']
    }
    holidays_df = pd.DataFrame(holidays)
    holidays_df = pd.melt(holidays_df)

    holidays_df.columns = ['Holiday','Date']
    return holidays_df


def get_holidays_near_features(df):
    # TODO Deixar parametrizado (coluna e tamanho janela)
    '''
    função que pega semanas antes do feriado
    '''
    
    df['Holiday_Christmas_2'] = df['Holiday_Christmas']\
                            + df['Holiday_Christmas'].shift(-1)\
                            + df['Holiday_Christmas'].shift(-2)\
                            + df['Holiday_Christmas'].shift(-3)
    df['Holiday_Christmas_2'] = df['Holiday_Christmas_2'].fillna(0)

    df['Holiday_Labor Day2'] = df['Holiday_Labor Day']\
                                + df['Holiday_Labor Day'].shift(-1)\
                                + df['Holiday_Labor Day'].shift(-2)\
                                + df['Holiday_Labor Day'].shift(-3)
    df['Holiday_Labor Day2'] = df['Holiday_Labor Day2'].fillna(0)

    df['Holiday_Super Bowl2'] = df['Holiday_Super Bowl']\
                                + df['Holiday_Super Bowl'].shift(-1)\
                                + df['Holiday_Super Bowl'].shift(-2)\
                                + df['Holiday_Super Bowl'].shift(-3)
    df['Holiday_Super Bowl2'] = df['Holiday_Super Bowl2'].fillna(0)

    df['Holiday_Thanksgiving2'] = df['Holiday_Thanksgiving']\
                                + df['Holiday_Thanksgiving'].shift(-1)\
                                + df['Holiday_Thanksgiving'].shift(-2)\
                                + df['Holiday_Thanksgiving'].shift(-3)
    df['Holiday_Thanksgiving2'] = df['Holiday_Thanksgiving2'].fillna(0)

    return df


def get_shift_sales(df, weeks):
    '''
    função que faz shift das vendas em N semanas e obem estes valores
    retorna o dataframe add tais dados em novas colunas
    '''
    # Pivot Data to get each Weekly_Sales in each Store/Dept by Date
    df_aux = pd.pivot_table(df, values='Weekly_Sales', index=['Date'],columns=['Store', 'Dept'])
    # Shifting all Weekly_Sales by N weeks and melting data
    df_aux = pd.melt(df_aux.shift(weeks).fillna(-1), ignore_index=False)
    df_aux.columns = ['Store','Dept','Weekly_Sales_aux']
    # Reseting index to get Data as collumn
    df_aux = df_aux.reset_index()
    df_aux = df.merge(df_aux, on = ['Date','Store','Dept'], how = 'left')
    return df_aux['Weekly_Sales_aux']


def linreg(Y):
    """
    funçao para achar a, b na função y = ax + b tal que a raiz da média
    da distância entre a linha de tendência e os pontos originais é minimizada    
    """
    X = range(len(Y))
    N = len(X)
    Sx = Sy = Sxx = Syy = Sxy = 0.0
    for x, y in zip(X, Y):
        Sx = Sx + x
        Sy = Sy + y
        Sxx = Sxx + x*x
        Syy = Syy + y*y
        Sxy = Sxy + x*y
    det = Sxx * N - Sx * Sx
    return [(Sxy * N - Sy * Sx)/det, (Sxx * Sy - Sx * Sxy)/det]


def get_regression_linear_pred(df, cols):
    '''
    fução que faz a reg linear nas colunas e retorna a, b e predict
    '''
    # Applying Linear Regression in last 2 years
    df_aux = df[cols].apply(lambda x: linreg(x.values),axis=1)
    df_aux = pd.DataFrame(df_aux.tolist())

    
    #Geting weekly Sales 1 Year Ago, a and b
    df_aux.columns = ['Weekly_Sales_1ya_a','Weekly_Sales_1ya_b']
    df_aux['Linear_Pred_1ya'] = (df_aux['Weekly_Sales_1ya_a'] * len(cols)) + df_aux['Weekly_Sales_1ya_b']
    
    return df_aux



def evaluate_model(y_test,y_pred):
    df_pred = pd.DataFrame({'y_true':y_test,'y_pred':y_pred})
    df_pred.plot()
    

## Criação de um modelo inicial (CRISP-DM)
* Com base na matriz de correlação, escolhi as primeiras features que entrarão no modelo inicial. Nos gráficos de barras de vendas por Loja e por Departamento já é possível ver que as vendas não estão igualmente distribuídas. Existe uma variação ali dependendo da loja/dept, mas seria o ideal criar 1 modelo para cada um dos 99 departamentos existentes?

In [ ]:
## Leitura, criação do dataframe unindo dados dos arquivos
filelist = [path+'/train.csv.zip',path+'/stores.csv',path+'/features.csv.zip']
df = read_create_merge_data(filelist)
display(df.head())


## Gera matriz de correlação
print('\n\n')
plot_heatmap(df)


## Gera gráfico de barras
print('\n\n')
plot_bar(df,'Store','Weekly_Sales')
plot_bar(df,'Dept','Weekly_Sales')


In [ ]:
## Seleção das colunas de interesse
data = df[['Store','Dept', 'IsHoliday', 'Size','Unemployment',"Weekly_Sales"]]

## seleção de features
X = data.drop(['Weekly_Sales'], axis=1)
## seleão do target
y = pd.DataFrame(data['Weekly_Sales'])

## separação em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
regr_model = create_rf_model(X,y)


In [ ]:
## TODO: Ver qual a melhor métrica a ser utilizada para avaliar o modelo
y_pred = regr_model.predict(X_test)

print("MAE" , metrics.mean_absolute_error(y_test, y_pred))
print("MSE" , metrics.mean_squared_error(y_test, y_pred))
print("RMSE" , np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("R2" , metrics.explained_variance_score(y_test, y_pred))
print("MAPE" ,metrics.mean_absolute_percentage_error(y_test, y_pred))



In [ ]:
serie = df[['Weekly_Sales']]
serie['pred'] = regr_model.predict(X)
serie.plot()


plt.figure()
feat_importances = pd.Series(regr_model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

## Feature Engineering e novo modelo

In [ ]:
## leitura dos dados
df = pd.read_csv(path+"/train.csv.zip")

## extração dia/ano/mes/semana do campo data e união de dataframes
df = treat_data(df)
df = df.merge(get_holidays_df(),how='left').fillna(0)


# One Hot Encode
df = df.join(pd.get_dummies(df['Holiday'], prefix='Holiday'))
df = df.drop('Holiday',axis=1)


## semanas antes do feriado
df = get_holidays_near_features(df)


## Tempo minimo de predição
df['Weekly_Sales_48'] = get_shift_sales(df, 48)
## 1 ano antes do tempo minimo para predição
df['Weekly_Sales_100'] = get_shift_sales(df, 100)
## 1 ano antes da semana a ser estimada
df['Weekly_Sales_52'] = get_shift_sales(df, 52)
## 2 anos antes da semana a ser estimada
df['Weekly_Sales_104'] = get_shift_sales(df, 104)


## regressão linear para achar pontos a, b e predict
df[['Weekly_Sales_1ya_a',
    'Weekly_Sales_1ya_b',
    'Linear_Pred_1ya']] = get_regression_linear_pred(df, ['Weekly_Sales_104','Weekly_Sales_52'])
df.head()


In [ ]:
%%time

## separa em treino e teste (1 ano para test)
df_train = df[df['Date'] < '2011-10-26']
df_test = df[df['Date'] >= '2011-10-26']

y_train = df_train['Weekly_Sales']
X_train = df_train.drop(['Weekly_Sales','Date'],axis=1)

y_test = df_test['Weekly_Sales']
X_test = df_test.drop(['Weekly_Sales','Date'],axis=1)

regr = RandomForestRegressor(random_state=0)
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

evaluate_model(y_test,y_pred)

In [ ]:
%%time
df_train2 = df_train[:90]
df_test2 = df_test[:53]

y_train2 = df_train2['Weekly_Sales']
X_train2 = df_train2.drop(['Weekly_Sales','Date'],axis=1)

y_test2 = df_test2['Weekly_Sales']
X_test2 = df_test2.drop(['Weekly_Sales','Date'],axis=1)

regr2 = RandomForestRegressor(random_state=0)
regr2.fit(X_train2, y_train2)

y_pred2 = regr2.predict(X_test2)
df_pred2 = pd.DataFrame({'y_true':y_test2.values,'y_pred':y_pred2})
df_pred2.plot()
df_pred2.index = y_test2.index

plt.figure()
df_plot = df_train2.append(df_test2)
df_plot['pred'] = df_pred2['y_pred']
df_plot[['Weekly_Sales','pred']].plot()

plt.figure()
feat_importances = pd.Series(regr2.feature_importances_, index=X_train.columns)
feat_importances.nlargest(30).plot(kind='barh')

In [ ]:
## leitura dos dados
df_sub = pd.read_csv(path+"/test.csv.zip")

## extração dia/ano/mes/semana do campo data e união de dataframes
df_sub = treat_data(df_sub)
df_sub = df_sub.merge(get_holidays_df(),how='left').fillna(0)
df_sub = df_sub.drop('Date', axis=1)

# One Hot Encode
df_sub = df_sub.join(pd.get_dummies(df_sub['Holiday'], prefix='Holiday'))
df_sub = df_sub.drop('Holiday',axis=1)
df_sub.head()

df_sub['Holiday_Labor Day'] = 0

## semanas antes do feriado
df_sub = get_holidays_near_features(df_sub)


## Tempo minimo de predição
df_sub['Weekly_Sales_48'] = 0
## 1 ano antes do tempo minimo para predição
df_sub['Weekly_Sales_100'] = 0
## 1 ano antes da semana a ser estimada
df_sub['Weekly_Sales_52'] = 0
## 2 anos antes da semana a ser estimada
df_sub['Weekly_Sales_104'] = 0


## regressão linear para achar pontos a, b e predict
df_sub[['Weekly_Sales_1ya_a',
    'Weekly_Sales_1ya_b',
    'Linear_Pred_1ya']] = get_regression_linear_pred(df_sub, ['Weekly_Sales_104','Weekly_Sales_52'])


df_sub.head()

In [ ]:
submission = pd.read_csv(path+'/sampleSubmission.csv.zip')
submission['Weekly_Sales']=regr2.predict(df_sub)
submission.head()

In [ ]:
submission.to_csv('submission_gac.csv')